In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib


# Загрузка данных из БД 
def load_data_from_db(db_path='tours.db'):
    try:
        conn = sqlite3.connect(db_path)
        query = """
        SELECT 
            id,
            name,
            price,
            dates,
            url,
            location,
            tour_type,
            rating,
            rating_count,
            image,
            description,
            created_at
        FROM tours
        """
        
        # датасет из бд
        df = pd.read_sql_query(query, conn)
        
        return df
    
    except sqlite3.Error as e:
        print(f"Ошибка при работе с SQLite: {e}")
        return pd.DataFrame() 
    
    finally:
        if 'conn' in locals():
            conn.close()

df = load_data_from_db()

# Предобработка данных
def preprocess_data(df):
    # Обработка цены
    def clean_price(price_str):
        if pd.isna(price_str):
            return np.nan

        cleaned = str(price_str).strip()
        cleaned = cleaned.replace('RUB', '').replace('€', '1').replace(' ', '')
        cleaned = cleaned.replace(',', '')

    
    df['price'] = df['price'].apply(clean_price)
    
    # Обработка даты получение даты отправки и обратно
    df['start_month'] = df['dates'].apply(lambda x: extract_month(x.split('—')[0].strip()))
    df['end_month'] = df['dates'].apply(lambda x: extract_month(x.split('—')[1].strip()) if '—' in x else np.nan)
    
    # Кодирование категориальных переменных
    cat_cols = ['location', 'tour_type']
    for col in cat_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].fillna('Unknown'))
    
    # Заполнение пропусков
    df.fillna({
        'rating': 0,
        'rating_count': 0,
        'start_month': df['start_month'].mode()[0],
        'end_month': df['end_month'].mode()[0],
        'price': df['price'].median()
    }, inplace=True)
    
    return df

def extract_month(date_str):
    month_map = {
        'января': 1, 'февраля': 2, 'марта': 3, 'апреля': 4,
        'мая': 5, 'июня': 6, 'июля': 7, 'августа': 8,
        'сентября': 9, 'октября': 10, 'ноября': 11, 'декабря': 12
    }
    for month_name, month_num in month_map.items():
        if month_name in date_str:
            return month_num
    return np.nan


In [ ]:
def define_target(df):
    df['optimal_purchase_month'] = df['start_month'] - 2
    df['optimal_purchase_month'] = df['optimal_purchase_month'].apply(lambda x: x + 12 if x < 1 else x)
    return df

In [24]:
def train_model(df):
    features = ['price', 'start_month', 'end_month', 'location', 'tour_type', 'rating', 'rating_count']
    target = 'optimal_purchase_month'
    
    X = df[features]
    y = df[target]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    print(classification_report(y_test, model.predict(X_test)))
    
    return model

df = preprocess_data(df)
df = define_target(df)
model = train_model(df)

# Сохранение модели
joblib.dump(model, 'tour_purchase_model.pkl')

              precision    recall  f1-score   support

         2.0       1.00      1.00      1.00       354
         3.0       1.00      1.00      1.00        13
         4.0       1.00      1.00      1.00        11
         5.0       1.00      1.00      1.00         5
         6.0       0.80      1.00      0.89         4
         7.0       0.00      0.00      0.00         1
        10.0       0.00      0.00      0.00         1

    accuracy                           0.99       389
   macro avg       0.69      0.71      0.70       389
weighted avg       0.99      0.99      0.99       389



c:\Users\Алексей\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Алексей\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Алексей\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

['tour_purchase_model.pkl']

In [52]:
import pandas as pd

def predict_for_all_tours(df, model):
    #модель предсказания оптимальных туров для покупки
    df_processed = preprocess_data(df.copy())
    features = ['price', 'start_month', 'end_month', 'location', 'tour_type', 'rating', 'rating_count']
    X = df_processed[features]
    df['optimal_purchase_month'] = model.predict(X)
    return df

if not df.empty:
    df_with_predictions = predict_for_all_tours(df, model)
    #print(df_with_predictions[['name', 'dates', 'optimal_purchase_month']])
    
def calculate_price_score(row):
    #расчет оптимальности цены по цене и рейтингу и кол-ву отзывов
    rating_score = float(row['rating']) / 5
    cout_rat = float(row['rating_count']) / 100
    price_score = 1 / (row['price'] + 1)
    return 0.9 * rating_score + 0.3 * price_score + 0.001 * cout_rat

def get_top_10_best_value_tours(df, model, dt=10):
    df = predict_for_all_tours(df, model)
    df['price_score'] = df.apply(calculate_price_score, axis=1)
    top_tours = df.sort_values('price_score', ascending=False).head(dt)
    result = top_tours[[
        'name', 
        'price', 
        'dates',
        'rating',
        'rating_count',
        'optimal_purchase_month',
        'price_score'
    ]]
    
    return result.reset_index(drop=True)

if not df.empty:
    # вместо 10 можно указать луюбое кол-во
    cou=10
    top_10_tours = get_top_10_best_value_tours(df, model, cou)
    
    print(f"Топ-{cou} туров по оптимальному сочетанию цены и качества:")
    print("-" * 80)
    
    for i, (_, row) in enumerate(top_10_tours.iterrows(), 1):
        print(f"{i}. {row['name']}")
        print(f"   Цена: {row['price']:.2f} RUB")
        print(f"   Даты: {row['dates']}")
        print(f"   Рейтинг: {row['rating']} (отзывов: {row['rating_count']})")
        print(f"   Оптимальный месяц покупки: {row['optimal_purchase_month']}")
        print(f"   Показатель оптимальности: {row['price_score']:.3f}")
        print("-" * 80)

Топ-10 туров по оптимальному сочетанию цены и качества:
--------------------------------------------------------------------------------
1. Очарование Ольхона
   Цена: 48700.00 RUB
   Даты: 18—23 мая
   Рейтинг: 5.0 (отзывов: 731)
   Оптимальный месяц покупки: 2.0
   Показатель оптимальности: 0.907
--------------------------------------------------------------------------------
2. Очарование Ольхона. Сокращенная программа
   Цена: 36900.00 RUB
   Даты: 18—22 мая
   Рейтинг: 5.0 (отзывов: 681)
   Оптимальный месяц покупки: 2.0
   Показатель оптимальности: 0.907
--------------------------------------------------------------------------------
3. Байкальский странник. Экскурсии по КБЖД и отдых на Ольхоне
   Цена: 79700.00 RUB
   Даты: 12—18 июня
   Рейтинг: 5.0 (отзывов: 661)
   Оптимальный месяц покупки: 2.0
   Показатель оптимальности: 0.907
--------------------------------------------------------------------------------
4. Знакомство с Байкалом. Познавательный тур
   Цена: 87700.00 RUB
